# Projection de patrimoine boursier

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime
import pandas as pd
import yaml

In [2]:
with open("config.yaml", "r") as f:
    config = yaml.safe_load(f)

MAX_APPORT_PEA = config["MAX_APPORT_PEA"]
DATA_FOLDER = config["DATA_FOLDER"]

In [3]:
effort_depargne = 2000
taux_dinteret = 0.12
nombre_danne = 6
age_initial = 26
date_actuelle = datetime.today()

In [4]:
patrimoine = pd.read_csv(DATA_FOLDER + "valorisation.csv")
patrimoine["Valorisation"] = pd.to_numeric(patrimoine["Valorisation"])
patrimoine_total = patrimoine["Valorisation"].sum()
valorisation_bourse = patrimoine.loc[
    patrimoine["Actif"] == "Action", "Valorisation"
].sum()

In [5]:
apport = pd.read_csv(DATA_FOLDER + "apport.csv")
apport["Date"] = pd.to_datetime(apport["Date"], dayfirst=True)
apport["Montant"] = pd.to_numeric(apport["Montant"])
apport_initial = apport.loc[apport["Actif"]=="Action","Montant"].sum()

In [ ]:
dates = pd.date_range(start=date_actuelle, periods=nombre_danne * 12, freq="ME")
print(len(dates))

In [7]:
def calcul_interet_composes_avec_apport(E, i, k0, mois):
    r = i / 12
    valo = k0 * np.power(1 + r, mois) + E * (np.power(1 + r, mois) - 1) / r
    return valo


def calcul_interet_composes(k0, i):
    r = i / 12
    valo = k0 * (1 + r)
    return valo

In [8]:
evolution = []
for mois in range(nombre_danne * 12):
    if apport_initial + effort_depargne * mois < MAX_APPORT_PEA:
        evolution.append(
            calcul_interet_composes_avec_apport(
                effort_depargne, taux_dinteret, valorisation_bourse, mois
            )
        )
    else:
        evolution.append(calcul_interet_composes(evolution[-1], taux_dinteret))

In [9]:
ages = [age_initial + i // 12 for i in range(len(dates))]

In [10]:
plt.figure(figsize=(14, 8))
plt.plot(dates, evolution, label="Evolution du capital Boursier")
plt.title("Évolution des intérêts composés avec apport mensuel sur PEA")
plt.xlabel("Date")
plt.ylabel("Montant (€)")
plt.grid()
plt.legend()

for i in range(0, len(dates), 12):  # Tous les ans
    plt.text(
        dates[i],
        evolution[i],
        f"{ages[i]} ans - {evolution[i]:,.2f}€",
        fontsize=9,
        ha="center",
        color="blue",
    )
# plt.text(
#     0,
#     -1.5,
#     f"Patrimoine final en {nombre_danne} ans : {evolution[-1]:,.2f}€",
#     ha="right",
#     fontsize=12,
#     color="black",
# )

ax = plt.gca()
ax.xaxis.set_major_formatter(mdates.DateFormatter("%b %y"))

plt.xticks(dates[::3], rotation=45)
plt.tight_layout()
plt.savefig("image/" + "projection.png")
# plt.show()
plt.close()